In [1]:
print("X")

X


In [2]:
%%time

import json
reviews = []
with open("yelp_academic_dataset_review.json") as f:
    index = 0
    for line in f:
        if index % 1000000 == 0:
            print(index)
        index += 1
        reviews.append(json.loads(line))

0
1000000
2000000
3000000
4000000
CPU times: user 43.1 s, sys: 5.39 s, total: 48.5 s
Wall time: 23min 52s


In [3]:
print(reviews[0])

{'useful': 0, 'date': '2011-10-10', 'type': 'review', 'review_id': 'NxL8SIC5yqOdnlXCg18IBg', 'text': "If you enjoy service by someone who is as competent as he is personable, I would recommend Corey Kaplan highly. The time he has spent here has been very productive and working with him educational and enjoyable. I hope not to need him again (though this is highly unlikely) but knowing he is there if I do is very nice. By the way, I'm not from El Centro, CA. but Scottsdale, AZ.", 'stars': 5, 'funny': 0, 'business_id': '2aFiy99vNLklCx3T_tGS9A', 'cool': 0, 'user_id': 'KpkOkG6RIf4Ra25Lhhxf1A'}


In [43]:
from collections import Counter
prolific_reviewers = Counter([review['user_id'] for review in reviews]).most_common(50)

In [44]:
keep_ids = {pr[0] : 0 for pr in prolific_reviewers}

In [45]:
by_author = {} # author : "review 1\n review 2\n review 3"
for review in reviews:
    uid = review['user_id']
    if uid in keep_ids:
        uid = review['user_id']
        if uid in by_author:
            by_author[uid] += "\n{}".format(review['text'])
        else:
            by_author[uid] = "{}".format(review['text'])


In [46]:
len(by_author)

50

In [47]:
# check that we have at least 200000 characters for each author
sorted([(len(by_author[key]), key) for key in by_author])[:5]

[(276136, 'ffPY_bHX8vLebHu8LBEqfg'),
 (278427, 'PeLGa5vUR8_mcsn-fn42Jg'),
 (338936, 'Q4Qfu-3vYtL1LRm2X1b0Gg'),
 (351311, 'cMEtAiW60I5wE_vLfTxoJQ'),
 (370129, 'iDlkZO2iILS8Jwfdy7DP9A')]

In [48]:
def get_chunks(l, n):
    n = max(1, n)
    return [l[i:i+n] for i in range(0, len(l), n)]

In [49]:

train_texts = []  # the first 100 000 chars from each author
train_labels = [] # each author
test_texts = []   # 100 texts of 1000 characters each (second 100 000 chars of each author)
test_labels = []  # each author * 100

author_int = {author: i for i,author in enumerate(by_author)}
int_author = {author_int[author]: author for author in author_int}

for author in by_author:
    train_text = by_author[author][:50000]
    train_label = author_int[author]
    train_texts.append(train_text)
    train_labels.append(train_label)
    
    short_texts = get_chunks(by_author[author][50000:100000], 1000)
    for text in short_texts:
        test_texts.append(text)
        test_labels.append(author_int[author])

In [50]:
data = {
    "train_texts": train_texts,
    "train_labels": train_labels,
    "test_texts": test_texts,
    "test_labels": test_labels
}

In [51]:
import pickle

with open("train_test_texts_labels.pickle", "wb") as f:
    pickle.dump(data, f)

In [52]:
import pickle

with open("train_test_texts_labels.pickle", "rb") as f:
    data = pickle.load(f)

In [53]:
train_texts = data['train_texts']
train_labels = data['train_labels']
test_texts = data['test_texts']
test_labels = data['test_labels']


In [54]:
print(len(train_texts), len(test_texts))

50 2500


In [55]:
len(train_texts[0])

50000

In [56]:
# vectorization - chars to ints
import string
import random
import sys

import numpy as np

from keras.models import load_model

def sample(preds, temperature=1.0):
    """Sample predictions from a probability array"""
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds + 1e-6) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def generate(model, diversity=0.5, text=""):
    """Generate text from a model"""
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated = ''
    sentence = text[start_index: start_index + maxlen]
    generated += sentence
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)

    for i in range(5000):
        x = np.zeros((1, maxlen), dtype=np.int)
        for t, char in enumerate(sentence):
            try:
                x[0, t] = char_indices[char]
            except:
                print(sentence)
        preds = model.predict(x, verbose=0)[0][0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]
        generated += next_char
        sentence = sentence[1:] + next_char
        sys.stdout.write(next_char)
        sys.stdout.flush()
    return

def vectorize(text):
    """Convert text into character sequences"""
    step = 3
    sentences = []
    next_chars = []
    for i in range(0, len(text) - maxlen, step):
        sentences.append(text[i: i + maxlen])
        next_chars.append(text[i + maxlen])
    X = np.zeros((len(sentences), maxlen), dtype=np.int)
    y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
    for i, sentence in enumerate(sentences):
        for t, char in enumerate(sentence):
            X[i, t] = char_indices[char]
        y[i, char_indices[next_chars[i]]] = 1
    return X, y

def clean_text(text, charset):
    text = " ".join(text.split())  # all white space is one space
    text = "".join([x for x in text if x in charset])  # remove characters that we don't care about
    return text

def get_model(modelfile, freeze=False):
    model = load_model(modelfile)
    if freeze:
        for layer in model.layers[:6]:
            layer.trainable = False
    return model

chars = " " + string.ascii_letters + string.punctuation  # sorted to keep indices consistent
charset = set(chars)  # for lookup
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

maxlen = 100  # must match length which generated model - the sequence length

# load a pretrained language model
modelfile = "charlm2/model_middlemarch_cnn.hdf5"

In [60]:
from keras.models import Sequential
from keras.layers import Embedding, Dropout, BatchNormalization, GRU, Dense

# https://blog.keras.io/using-pre-trained-word-embeddings-in-a-keras-model.html
print('Processing pretrained character embeds...')
embedding_vectors = {}
with open("glove.840B.300d-char.txt", "r") as f:
    for line in f:
        line_split = line.strip().split(" ")
        vec = np.array(line_split[1:], dtype=float)
        char = line_split[0]
        embedding_vectors[char] = vec

embedding_matrix = np.zeros((len(chars), 300))
#embedding_matrix = np.random.uniform(-1, 1, (len(chars), 300))
for char, i in char_indices.items():
    #print ("{}, {}".format(char, i))
    embedding_vector = embedding_vectors.get(char)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

def get_gru_model(use_embeddings=False):
    model = Sequential()
    if use_embeddings:
        model.add(Embedding(input_dim=len(charset), output_dim=300, weights=[embedding_matrix]))
    else:
        model.add(Embedding(input_dim=len(charset), output_dim=300))
    model.add(Dropout(0.1))
    model.add(BatchNormalization())
    model.add(GRU(256))
    model.add(Dropout(0.3))
    model.add(BatchNormalization())
    model.add(Dense(85, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    return model

Processing pretrained character embeds...


In [61]:
%%time
test_model = get_gru_model()
X, y = vectorize(clean_text(train_texts[0], charset))
test_model.fit(X, y, epochs=10, batch_size=128, validation_split=0.1)

Train on 14831 samples, validate on 1648 samples
Epoch 1/10
14831/14831 [==============================] - 19s - loss: 3.1480 - val_loss: 3.9208
Epoch 2/10
14831/14831 [==============================] - 16s - loss: 2.4809 - val_loss: 3.2844
Epoch 3/10
14831/14831 [==============================] - 17s - loss: 2.2282 - val_loss: 2.7332
Epoch 4/10
14831/14831 [==============================] - 16s - loss: 2.0630 - val_loss: 2.3528
Epoch 5/10
14831/14831 [==============================] - 16s - loss: 1.9500 - val_loss: 2.1056
Epoch 6/10
14831/14831 [==============================] - 16s - loss: 1.8630 - val_loss: 2.0248
Epoch 7/10
14831/14831 [==============================] - 16s - loss: 1.7832 - val_loss: 2.0104
Epoch 8/10
14831/14831 [==============================] - 16s - loss: 1.7099 - val_loss: 2.0018
Epoch 9/10
14831/14831 [==============================] - 16s - loss: 1.6386 - val_loss: 1.9920
Epoch 10/10
14831/14831 [==============================] - 16s - loss: 1.5568 - val_los

In [ ]:
%%time
author_models = []  # [(author_model, author_id), (author_model, author_id), ...] - ids are ints
for i, train_text in enumerate(train_texts):
    print("{} / {}".format(i, len(train_texts)))
    ct = clean_text(train_text, charset)
    am = get_gru_model()
    X, y = vectorize(ct)
    am.fit(X, y, epochs=10, batch_size=128, verbose=0)
    author_models.append((am, train_labels[i]))

0 / 50
1 / 50
2 / 50
3 / 50
4 / 50
5 / 50
6 / 50
7 / 50
8 / 50
9 / 50
10 / 50
11 / 50
12 / 50
13 / 50
14 / 50
16 / 50
17 / 50
18 / 50
19 / 50
20 / 50
21 / 50
22 / 50
23 / 50
24 / 50
25 / 50
26 / 50
27 / 50
28 / 50
29 / 50
30 / 50
31 / 50
32 / 50
33 / 50
34 / 50
35 / 50
36 / 50
37 / 50
38 / 50
39 / 50
40 / 50
41 / 50
42 / 50
43 / 50
44 / 50
45 / 50
46 / 50
47 / 50
48 / 50
49 / 50
CPU times: user 3h 45min 50s, sys: 17min 12s, total: 4h 3min 2s
Wall time: 2h 35min 14s


In [64]:
print(1)

1


In [63]:
print(len(author_models))
print(len(test_texts))
print(len(test_labels))

50
2500
2500


In [65]:
from statistics import mean
word_counts = [text.count(" ") for text in test_texts]
mean(word_counts)

182.5832

In [66]:
# 182 words is quite short
# Try to join 5 tests texts together
longer_test_texts = get_chunks(test_texts, 5)
longer_test_labels = get_chunks(test_labels, 5)

In [67]:
all([len(set(x)) == 1 for x in longer_test_labels])  # Make sure that all combined labels are the same

True

In [68]:
longer_test_texts = ['\n'.join(chunk) for chunk in longer_test_texts]

In [69]:
longer_test_labels = [chunk[0] for chunk in longer_test_labels]

In [70]:
len(longer_test_texts)

500

In [71]:
%%time
from random import shuffle
from datetime import datetime

def get_predictions(author_models, test_texts, test_labels):
    """Evaluate each text for each author_model and append first metric to predictions"""
    indicies = list(range(len(test_texts)))

    test_texts = np.array(test_texts)
    test_labels = np.array(test_labels)

    test_texts = test_texts[indicies]
    test_labels = test_labels[indicies]

    predictions = []
    for i, text in enumerate(test_texts):
        t1 = datetime.now()
        print("{} / {}".format(i, len(test_texts)), end=" ")
        X, y = vectorize(clean_text(text, charset))

        losses = []
        for am in author_models:
            print(".", end="")
            model = am[0]
            label = am[1]
            loss = model.evaluate(X, y, verbose=0)
            losses.append((loss, label))
        print(" {}".format(datetime.now() - t1))
        predictions.append(losses)
    return predictions
    

predictions_long = get_predictions(author_models, longer_test_texts, longer_test_labels)

0 / 500 .................................................. 0:04:53.823562
1 / 500 .................................................. 0:01:29.357596
2 / 500 .................................................. 0:01:29.403903
3 / 500 .................................................. 0:01:29.457545
4 / 500 .................................................. 0:01:29.418732
5 / 500 .................................................. 0:01:29.402586
6 / 500 .................................................. 0:01:29.338355
7 / 500 .................................................. 0:01:29.299863
8 / 500 .................................................. 0:01:29.201831
9 / 500 .................................................. 0:01:29.215665
10 / 500 .................................................. 0:01:29.218467
11 / 500 .................................................. 0:01:29.156110
12 / 500 .................................................. 0:01:29.178234
13 / 500 ..........................

109 / 500 .................................................. 0:01:31.387294
110 / 500 .................................................. 0:01:29.615783
111 / 500 .................................................. 0:01:31.243825
112 / 500 .................................................. 0:01:31.158846
113 / 500 .................................................. 0:01:31.100170
114 / 500 .................................................. 0:01:31.254315
115 / 500 .................................................. 0:01:31.249152
116 / 500 .................................................. 0:01:31.238190
117 / 500 .................................................. 0:01:31.247292
118 / 500 .................................................. 0:01:31.156884
119 / 500 .................................................. 0:01:29.593372
120 / 500 .................................................. 0:01:31.333271
121 / 500 .................................................. 0:01:31.355232
122 / 500 ..

217 / 500 .................................................. 0:01:29.283050
218 / 500 .................................................. 0:01:29.267942
219 / 500 .................................................. 0:01:29.236104
220 / 500 .................................................. 0:01:29.339491
221 / 500 .................................................. 0:01:29.303076
222 / 500 .................................................. 0:01:29.292002
223 / 500 .................................................. 0:01:29.386147
224 / 500 .................................................. 0:01:29.345570
225 / 500 .................................................. 0:01:29.303094
226 / 500 .................................................. 0:01:29.352727
227 / 500 .................................................. 0:01:29.399909
228 / 500 .................................................. 0:01:29.314526
229 / 500 .................................................. 0:01:29.354569
230 / 500 ..

325 / 500 .................................................. 0:01:29.344109
326 / 500 .................................................. 0:01:29.301312
327 / 500 .................................................. 0:01:29.301086
328 / 500 .................................................. 0:01:29.316883
329 / 500 .................................................. 0:01:29.308441
330 / 500 .................................................. 0:01:29.284104
331 / 500 .................................................. 0:01:29.326629
332 / 500 .................................................. 0:01:29.258847
333 / 500 .................................................. 0:01:29.319286
334 / 500 .................................................. 0:01:29.292448
335 / 500 .................................................. 0:01:29.359710
336 / 500 .................................................. 0:01:29.329391
337 / 500 .................................................. 0:01:29.306317
338 / 500 ..

433 / 500 .................................................. 0:01:27.590447
434 / 500 .................................................. 0:01:29.202817
435 / 500 .................................................. 0:01:29.217068
436 / 500 .................................................. 0:01:29.159146
437 / 500 .................................................. 0:01:29.212964
438 / 500 .................................................. 0:01:29.202807
439 / 500 .................................................. 0:01:29.190238
440 / 500 .................................................. 0:01:29.321917
441 / 500 .................................................. 0:01:29.324443
442 / 500 .................................................. 0:01:29.339277
443 / 500 .................................................. 0:01:29.297215
444 / 500 .................................................. 0:01:29.357958
445 / 500 .................................................. 0:01:29.317098
446 / 500 ..

In [72]:
len(predictions_long)

500

In [73]:
print("X")

X


In [74]:
pred_is = []
for pred in predictions_long:
    pred_i = [p[0] for p in pred]
    pred_is.append(pred_i)

In [75]:
pred_labs = [np.argmin(pred) for pred in pred_is]

In [76]:
from sklearn.metrics import accuracy_score
accuracy_score(longer_test_labels, pred_labs)

0.90000000000000002

In [ ]:
test_author_model.summary()

In [ ]:
test_author_model.evaluatei

In [ ]:
len(train_texts)

In [ ]:
len(train_labels)

In [116]:
from sklearn.pipeline import FeatureUnion
from sklearn.feature_extraction.text import TfidfVectorizer
word_vec = TfidfVectorizer(min_df=5, ngram_range=(1,2), lowercase=False)
char_vec = TfidfVectorizer(min_df=5, ngram_range=(2,3), lowercase=False)

fu = FeatureUnion([
    ('word', word_vec),
    ('char', char_vec)
])


X_train = fu.fit_transform(train_texts)

In [117]:
X_test = fu.transform(test_texts)

In [118]:
from sklearn.svm import LinearSVC

svm = LinearSVC()

In [119]:
svm.fit(X_train, train_labels)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [120]:
preds = svm.predict(X_test)

In [121]:
accuracy_score(test_labels, preds)

0.44600000000000001

In [122]:
X_test_longer = fu.transform(longer_test_texts)

In [123]:
preds = svm.predict(X_test_longer)

In [124]:
accuracy_score(longer_test_labels, preds)

0.88400000000000001

In [114]:
X.shape

(16400, 100)

In [115]:
X, y = vectorize(clean_text(train_texts[3], charset))

In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
y[0]

In [ ]:
X[0]

In [ ]:
test_author_model.layers[:6]

In [37]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM, Input, Embedding, Conv1D, MaxPooling1D, BatchNormalization, GRU
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

# load ascii text and covert to lowercase

# cnn = Dropout(0.2)(embedded)
# cnn = Conv1D(128, 5, activation='relu')(cnn)
# cnn = MaxPooling1D(pool_size=4)(cnn)

model = Sequential()
model.add(Embedding(input_dim=len(charset), output_dim=100))
model.add(Dropout(0.1))
model.add(BatchNormalization())
model.add(GRU(256))
model.add(Dropout(0.3))
model.add(BatchNormalization())
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.fit(X, y, epochs=13, batch_size=128, validation_split=0.1)

"""
LSTM, BatchNorm
Train on 14929 samples, validate on 1659 samples
Epoch 1/5
14929/14929 [==============================] - 75s - loss: 3.0403 - val_loss: 3.4757
Epoch 2/5
14929/14929 [==============================] - 60s - loss: 2.3156 - val_loss: 2.9687

LSTM
Train on 14929 samples, validate on 1659 samples
Epoch 1/5
14929/14929 [==============================] - 75s - loss: 3.1259 - val_loss: 2.7865
Epoch 2/5
14929/14929 [==============================] - 60s - loss: 2.6150 - val_loss: 2.3894

CNN(5), LSTM  # faster, needs more epochs
Train on 14929 samples, validate on 1659 samples
Epoch 1/5
14929/14929 [==============================] - 42s - loss: 3.1579 - val_loss: 2.9987
Epoch 2/5
14929/14929 [==============================] - 26s - loss: 2.8874 - val_loss: 2.6994
Epoch 3/5
14929/14929 [==============================] - 26s - loss: 2.6220 - val_loss: 2.4879
Epoch 4/5
14929/14929 [==============================] - 26s - loss: 2.4309 - val_loss: 2.3942
Epoch 5/5
14929/14929 [==============================] - 26s - loss: 2.2950 - val_loss: 2.2902

CNN(5), CNN(3), LSTM doesn't drop below 3.0 in 5 epochs


Embedding, BatchNorm, GRU, BatchNorm
Train on 14725 samples, validate on 1637 samples
Epoch 1/20
14725/14725 [==============================] - 16s - loss: 2.9240 - val_loss: 3.9516
Epoch 2/20
14725/14725 [==============================] - 15s - loss: 2.2447 - val_loss: 3.3667
Epoch 3/20
14725/14725 [==============================] - 15s - loss: 2.0054 - val_loss: 2.8011
Epoch 4/20
14725/14725 [==============================] - 15s - loss: 1.8388 - val_loss: 2.3477
Epoch 5/20
14725/14725 [==============================] - 15s - loss: 1.7122 - val_loss: 2.0196
Epoch 6/20
14725/14725 [==============================] - 15s - loss: 1.6069 - val_loss: 1.9417
Epoch 7/20
14725/14725 [==============================] - 15s - loss: 1.5044 - val_loss: 1.9541
Epoch 8/20
14725/14725 [==============================] - 15s - loss: 1.3987 - val_loss: 1.9512
Epoch 9/20
14725/14725 [==============================] - 15s - loss: 1.2940 - val_loss: 1.9921
Epoch 10/20
14725/14725 [==============================] - 15s - loss: 1.1850 - val_loss: 2.0424

Train on 14725 samples, validate on 1637 samples
Epoch 1/20
14725/14725 [==============================] - 17s - loss: 3.2577 - val_loss: 3.8892
Epoch 2/20
14725/14725 [==============================] - 15s - loss: 2.5444 - val_loss: 3.2364
Epoch 3/20
14725/14725 [==============================] - 15s - loss: 2.2915 - val_loss: 2.7205
Epoch 4/20
14725/14725 [==============================] - 15s - loss: 2.1258 - val_loss: 2.3477
Epoch 5/20
14725/14725 [==============================] - 15s - loss: 2.0219 - val_loss: 2.0588
Epoch 6/20
14725/14725 [==============================] - 15s - loss: 1.9386 - val_loss: 1.9478
Epoch 7/20
14725/14725 [==============================] - 15s - loss: 1.8762 - val_loss: 1.9152
Epoch 8/20
14725/14725 [==============================] - 15s - loss: 1.7994 - val_loss: 1.9076
Epoch 9/20
14725/14725 [==============================] - 15s - loss: 1.7224 - val_loss: 1.8829
Epoch 10/20
14725/14725 [==============================] - 15s - loss: 1.6783 - val_loss: 1.9007
Epoch 11/20
14725/14725 [==============================] - 15s - loss: 1.6154 - val_loss: 1.8910
Epoch 12/20
14725/14725 [==============================] - 15s - loss: 1.5485 - val_loss: 1.8920
Epoch 13/20
14725/14725 [==============================] - 15s - loss: 1.4844 - val_loss: 1.9198
Epoch 14/20
14725/14725 [==============================] - 15s - loss: 1.4291 - val_loss: 1.9193
Epoch 15/20
14725/14725 [==============================] - 15s - loss: 1.3670 - val_loss: 1.9295
Epoch 16/20
14725/14725 [==============================] - 15s - loss: 1.3028 - val_loss: 1.9752
Epoch 17/20
14725/14725 [==============================] - 15s - loss: 1.2321 - val_loss: 1.9969
Epoch 18/20
14725/14725 [==============================] - 15s - loss: 1.2159 - val_loss: 2.0431
Epoch 19/20
14725/14725 [==============================] - 15s - loss: 1.1391 - val_loss: 2.0709
Epoch 20/20
14725/14725 [==============================] - 15s - loss: 1.1321 - val_loss: 2.1172


model = Sequential()
model.add(Embedding(input_dim=len(charset), output_dim=100))
model.add(BatchNormalization())
model.add(GRU(256))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.fit(X, y, epochs=20, batch_size=128, validation_split=0.1)
Train on 14725 samples, validate on 1637 samples
Epoch 1/20
14725/14725 [==============================] - 17s - loss: 3.2577 - val_loss: 3.8892
Epoch 2/20
14725/14725 [==============================] - 15s - loss: 2.5444 - val_loss: 3.2364
Epoch 3/20
14725/14725 [==============================] - 15s - loss: 2.2915 - val_loss: 2.7205
Epoch 4/20
14725/14725 [==============================] - 15s - loss: 2.1258 - val_loss: 2.3477
Epoch 5/20
14725/14725 [==============================] - 15s - loss: 2.0219 - val_loss: 2.0588
Epoch 6/20
14725/14725 [==============================] - 15s - loss: 1.9386 - val_loss: 1.9478
Epoch 7/20
14725/14725 [==============================] - 15s - loss: 1.8762 - val_loss: 1.9152
Epoch 8/20
14725/14725 [==============================] - 15s - loss: 1.7994 - val_loss: 1.9076
Epoch 9/20
14725/14725 [==============================] - 15s - loss: 1.7224 - val_loss: 1.8829
Epoch 10/20
14725/14725 [==============================] - 15s - loss: 1.6783 - val_loss: 1.9007
Epoch 11/20
14725/14725 [==============================] - 15s - loss: 1.6154 - val_loss: 1.8910
Epoch 12/20
14725/14725 [==============================] - 15s - loss: 1.5485 - val_loss: 1.8920
Epoch 13/20
14725/14725 [==============================] - 15s - loss: 1.4844 - val_loss: 1.9198
Epoch 14/20
14725/14725 [==============================] - 15s - loss: 1.4291 - val_loss: 1.9193
Epoch 15/20
14725/14725 [==============================] - 15s - loss: 1.3670 - val_loss: 1.9295
Epoch 16/20
14725/14725 [==============================] - 15s - loss: 1.3028 - val_loss: 1.9752
Epoch 17/20
14725/14725 [==============================] - 15s - loss: 1.2321 - val_loss: 1.9969
Epoch 18/20
14725/14725 [==============================] - 15s - loss: 1.2159 - val_loss: 2.0431
Epoch 19/20
14725/14725 [==============================] - 15s - loss: 1.1391 - val_loss: 2.0709
Epoch 20/20
14725/14725 [==============================] - 15s - loss: 1.1321 - val_loss: 2.1172


model = Sequential()
model.add(Embedding(input_dim=len(charset), output_dim=100))
model.add(GRU(256))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.fit(X, y, epochs=20, batch_size=128, validation_split=0.1)
Train on 14725 samples, validate on 1637 samples
Epoch 1/20
14725/14725 [==============================] - 16s - loss: 3.1731 - val_loss: 3.5648
Epoch 2/20
14725/14725 [==============================] - 14s - loss: 2.4964 - val_loss: 2.9875
Epoch 3/20
14725/14725 [==============================] - 14s - loss: 2.3446 - val_loss: 2.7695
Epoch 4/20
14725/14725 [==============================] - 14s - loss: 2.2928 - val_loss: 2.6010
Epoch 5/20
14725/14725 [==============================] - 14s - loss: 2.2642 - val_loss: 2.3900
Epoch 6/20
14725/14725 [==============================] - 14s - loss: 2.2373 - val_loss: 2.5023
Epoch 7/20
14725/14725 [==============================] - 14s - loss: 2.2186 - val_loss: 2.3780
Epoch 8/20
14725/14725 [==============================] - 14s - loss: 2.2029 - val_loss: 2.4928
Epoch 9/20
14725/14725 [==============================] - 14s - loss: 2.1852 - val_loss: 2.3480
Epoch 10/20
14725/14725 [==============================] - 14s - loss: 2.1745 - val_loss: 2.4801
Epoch 11/20
14725/14725 [==============================] - 14s - loss: 2.1563 - val_loss: 2.3951
Epoch 12/20
14725/14725 [==============================] - 14s - loss: 2.1391 - val_loss: 2.4133
Epoch 13/20
14725/14725 [==============================] - 14s - loss: 2.1192 - val_loss: 2.5896
Epoch 14/20
14725/14725 [==============================] - 14s - loss: 2.1020 - val_loss: 2.2692
Epoch 15/20
14725/14725 [==============================] - 14s - loss: 2.0770 - val_loss: 2.2179
Epoch 16/20
14725/14725 [==============================] - 14s - loss: 2.0643 - val_loss: 2.2822
Epoch 17/20
 6784/14725 [============>.................] - ETA: 7s - loss: 2.0302


model = Sequential()
model.add(Embedding(input_dim=len(charset), output_dim=100))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(GRU(256))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.fit(X, y, epochs=20, batch_size=128, validation_split=0.1)
Train on 14725 samples, validate on 1637 samples
Epoch 1/20
14725/14725 [==============================] - 17s - loss: 3.1528 - val_loss: 3.9042
Epoch 2/20
14725/14725 [==============================] - 15s - loss: 2.4658 - val_loss: 3.2093
Epoch 3/20
14725/14725 [==============================] - 15s - loss: 2.2001 - val_loss: 2.6764
Epoch 4/20
14725/14725 [==============================] - 15s - loss: 2.0358 - val_loss: 2.2919
Epoch 5/20
14725/14725 [==============================] - 15s - loss: 1.9497 - val_loss: 2.0060
Epoch 6/20
14725/14725 [==============================] - 15s - loss: 1.8588 - val_loss: 1.9313
Epoch 7/20
14725/14725 [==============================] - 15s - loss: 1.7883 - val_loss: 1.9153
Epoch 8/20
14725/14725 [==============================] - 15s - loss: 1.7034 - val_loss: 1.9145
Epoch 9/20
14725/14725 [==============================] - 15s - loss: 1.6382 - val_loss: 1.8979
Epoch 10/20
14725/14725 [==============================] - 15s - loss: 1.5827 - val_loss: 1.8864
Epoch 11/20
14725/14725 [==============================] - 15s - loss: 1.5093 - val_loss: 1.8967
Epoch 12/20
14725/14725 [==============================] - 15s - loss: 1.4472 - val_loss: 1.9040
Epoch 13/20
14725/14725 [==============================] - 15s - loss: 1.3809 - val_loss: 1.9227
Epoch 14/20
14725/14725 [==============================] - 15s - loss: 1.3225 - val_loss: 1.9469
Epoch 15/20
14725/14725 [==============================] - 15s - loss: 1.2516 - val_loss: 1.9862
Epoch 16/20
14725/14725 [==============================] - 15s - loss: 1.2094 - val_loss: 1.9963
Epoch 17/20
14725/14725 [==============================] - 15s - loss: 1.1658 - val_loss: 2.0331
Epoch 18/20
14725/14725 [==============================] - 15s - loss: 1.0851 - val_loss: 2.0452
Epoch 19/20
14725/14725 [==============================] - 15s - loss: 1.0394 - val_loss: 2.0810
Epoch 20/20
14725/14725 [==============================] - 15s - loss: 0.9903 - val_loss: 2.1283


model = Sequential()
model.add(Embedding(input_dim=len(charset), output_dim=100))
model.add(Dropout(0.3))
model.add(BatchNormalization())
model.add(GRU(256))
model.add(Dropout(0.3))
model.add(BatchNormalization())
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.fit(X, y, epochs=20, batch_size=128, validation_split=0.1)
Train on 14725 samples, validate on 1637 samples
Epoch 1/20
14725/14725 [==============================] - 17s - loss: 3.2991 - val_loss: 3.8902
Epoch 2/20
14725/14725 [==============================] - 15s - loss: 2.5672 - val_loss: 3.1627
Epoch 3/20
14725/14725 [==============================] - 15s - loss: 2.2731 - val_loss: 2.6340
Epoch 4/20
14725/14725 [==============================] - 15s - loss: 2.1316 - val_loss: 2.2594
Epoch 5/20
14725/14725 [==============================] - 15s - loss: 2.0249 - val_loss: 2.0159
Epoch 6/20
14725/14725 [==============================] - 15s - loss: 1.9571 - val_loss: 1.9456
Epoch 7/20
14725/14725 [==============================] - 15s - loss: 1.8789 - val_loss: 1.9213
Epoch 8/20
14725/14725 [==============================] - 15s - loss: 1.8233 - val_loss: 1.8924
Epoch 9/20
14725/14725 [==============================] - 15s - loss: 1.7575 - val_loss: 1.8987



model = Sequential()
model.add(Embedding(input_dim=len(charset), output_dim=100))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(GRU(256))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.fit(X, y, epochs=20, batch_size=128, validation_split=0.1)
Train on 14725 samples, validate on 1637 samples
Epoch 1/20
14725/14725 [==============================] - 17s - loss: 3.6036 - val_loss: 3.7924
Epoch 2/20
14725/14725 [==============================] - 15s - loss: 2.7765 - val_loss: 3.0022
Epoch 3/20
14725/14725 [==============================] - 15s - loss: 2.4773 - val_loss: 2.5697
Epoch 4/20
14725/14725 [==============================] - 15s - loss: 2.3218 - val_loss: 2.2606
Epoch 5/20
14725/14725 [==============================] - 15s - loss: 2.2328 - val_loss: 2.0832
Epoch 6/20
14725/14725 [==============================] - 15s - loss: 2.1748 - val_loss: 2.0248
Epoch 7/20
14725/14725 [==============================] - 15s - loss: 2.1174 - val_loss: 1.9865
Epoch 8/20
14725/14725 [==============================] - 15s - loss: 2.0617 - val_loss: 1.9640
Epoch 9/20
14725/14725 [==============================] - 15s - loss: 2.0206 - val_loss: 1.9461
Epoch 10/20
14725/14725 [==============================] - 15s - loss: 1.9758 - val_loss: 1.9334
Epoch 11/20
14725/14725 [==============================] - 15s - loss: 1.9546 - val_loss: 1.9148
Epoch 12/20
14725/14725 [==============================] - 15s - loss: 1.9045 - val_loss: 1.9121
Epoch 13/20
14725/14725 [==============================] - 15s - loss: 1.8757 - val_loss: 1.8888
Epoch 14/20
14725/14725 [==============================] - 15s - loss: 1.8437 - val_loss: 1.8874
Epoch 15/20
14725/14725 [==============================] - 15s - loss: 1.8145 - val_loss: 1.8822
Epoch 16/20
14725/14725 [==============================] - 15s - loss: 1.7805 - val_loss: 1.8785
Epoch 17/20
14725/14725 [==============================] - 15s - loss: 1.7558 - val_loss: 1.8868
Epoch 18/20
14725/14725 [==============================] - 15s - loss: 1.7218 - val_loss: 1.8670
Epoch 19/20
14725/14725 [==============================] - 15s - loss: 1.7032 - val_loss: 1.8759
Epoch 20/20
14725/14725 [==============================] - 15s - loss: 1.6832 - val_loss: 1.8834


model = Sequential()
model.add(Embedding(input_dim=len(charset), output_dim=100))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(GRU(256, return_sequences=True))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(GRU(256))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.fit(X, y, epochs=20, batch_size=128, validation_split=0.1)
Train on 14725 samples, validate on 1637 samples
Epoch 1/20
14725/14725 [==============================] - 33s - loss: 3.7814 - val_loss: 3.7282
Epoch 2/20
14725/14725 [==============================] - 29s - loss: 2.8453 - val_loss: 2.8542
Epoch 3/20
14725/14725 [==============================] - 29s - loss: 2.5178 - val_loss: 2.4434
Epoch 4/20
14725/14725 [==============================] - 29s - loss: 2.3762 - val_loss: 2.1894
Epoch 5/20
14725/14725 [==============================] - 29s - loss: 2.2896 - val_loss: 2.0862
Epoch 6/20
14725/14725 [==============================] - 29s - loss: 2.2254 - val_loss: 2.0516
Epoch 7/20
14725/14725 [==============================] - 29s - loss: 2.1565 - val_loss: 2.0133
Epoch 8/20
14725/14725 [==============================] - 29s - loss: 2.1132 - val_loss: 1.9992
Epoch 9/20
14725/14725 [==============================] - 29s - loss: 2.0798 - val_loss: 1.9881
Epoch 10/20
14725/14725 [==============================] - 29s - loss: 2.0509 - val_loss: 1.9784
Epoch 11/20
14725/14725 [==============================] - 29s - loss: 2.0198 - val_loss: 1.9618
Epoch 12/20
14725/14725 [==============================] - 29s - loss: 1.9822 - val_loss: 1.9383
Epoch 13/20
14725/14725 [==============================] - 29s - loss: 1.9437 - val_loss: 1.9300
Epoch 14/20
14725/14725 [==============================] - 29s - loss: 1.9198 - val_loss: 1.9163
Epoch 15/20
14725/14725 [==============================] - 29s - loss: 1.8989 - val_loss: 1.9160
Epoch 16/20
14725/14725 [==============================] - 29s - loss: 1.8866 - val_loss: 1.9085
Epoch 17/20
14725/14725 [==============================] - 29s - loss: 1.8493 - val_loss: 1.8965
Epoch 18/20
14725/14725 [==============================] - 29s - loss: 1.8248 - val_loss: 1.8878
Epoch 19/20
14725/14725 [==============================] - 29s - loss: 1.8037 - val_loss: 1.8870
Epoch 20/20
14725/14725 [==============================] - 29s - loss: 1.7724 - val_loss: 1.8862


model = Sequential()
model.add(Embedding(input_dim=len(charset), output_dim=100))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(GRU(256, return_sequences=True))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(GRU(256))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.fit(X, y, epochs=20, batch_size=128, validation_split=0.1)
Train on 14725 samples, validate on 1637 samples
Epoch 1/20
14725/14725 [==============================] - 32s - loss: 3.2809 - val_loss: 3.7595
Epoch 2/20
14725/14725 [==============================] - 29s - loss: 2.4379 - val_loss: 2.9869
Epoch 3/20
14725/14725 [==============================] - 29s - loss: 2.1504 - val_loss: 2.5361
Epoch 4/20
14725/14725 [==============================] - 29s - loss: 1.9887 - val_loss: 2.1294
Epoch 5/20
14725/14725 [==============================] - 29s - loss: 1.8984 - val_loss: 1.9727
Epoch 6/20
14725/14725 [==============================] - 29s - loss: 1.7892 - val_loss: 1.9264
Epoch 7/20
14725/14725 [==============================] - 29s - loss: 1.7172 - val_loss: 1.9100
Epoch 8/20
14725/14725 [==============================] - 29s - loss: 1.6361 - val_loss: 1.9124
Epoch 9/20
14725/14725 [==============================] - 29s - loss: 1.5621 - val_loss: 1.9122
Epoch 10/20
14725/14725 [==============================] - 29s - loss: 1.4863 - val_loss: 1.9045
Epoch 11/20
14725/14725 [==============================] - 29s - loss: 1.4150 - val_loss: 1.9278
Epoch 12/20
14725/14725 [==============================] - 29s - loss: 1.3691 - val_loss: 1.9181
Epoch 13/20
14725/14725 [==============================] - 29s - loss: 1.2970 - val_loss: 1.9414
Epoch 14/20
 1536/14725 [==>...........................] - ETA: 25s - loss: 1.1475
 
 
 
model = Sequential()
model.add(Embedding(input_dim=len(charset), output_dim=100))
model.add(BatchNormalization())
model.add(GRU(256))
model.add(Dropout(0.4))
model.add(BatchNormalization())
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.fit(X, y, epochs=20, batch_size=128, validation_split=0.1)
 Train on 14725 samples, validate on 1637 samples
Epoch 1/20
14725/14725 [==============================] - 17s - loss: 3.1787 - val_loss: 3.9282
Epoch 2/20
14725/14725 [==============================] - 15s - loss: 2.5070 - val_loss: 3.2479
Epoch 3/20
14725/14725 [==============================] - 15s - loss: 2.2191 - val_loss: 2.7522
Epoch 4/20
14725/14725 [==============================] - 15s - loss: 2.0637 - val_loss: 2.3331
Epoch 5/20
14725/14725 [==============================] - 15s - loss: 1.9539 - val_loss: 2.0326
Epoch 6/20
14725/14725 [==============================] - 15s - loss: 1.8622 - val_loss: 1.9440
Epoch 7/20
14725/14725 [==============================] - 15s - loss: 1.7821 - val_loss: 1.9166
Epoch 8/20
14725/14725 [==============================] - 15s - loss: 1.7169 - val_loss: 1.8996
Epoch 9/20
14725/14725 [==============================] - 15s - loss: 1.6561 - val_loss: 1.8849
Epoch 10/20
14725/14725 [==============================] - 15s - loss: 1.5910 - val_loss: 1.9032
Epoch 11/20
14725/14725 [==============================] - 15s - loss: 1.5082 - val_loss: 1.8878
Epoch 12/20
14725/14725 [==============================] - 15s - loss: 1.4513 - val_loss: 1.9252
Epoch 13/20
 7552/14725 [==============>...............] - ETA: 7s - loss: 1.3534
 
 
model = Sequential()
model.add(Embedding(input_dim=len(charset), output_dim=100))
model.add(BatchNormalization())
model.add(GRU(512))
model.add(Dropout(0.4))
model.add(BatchNormalization())
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.fit(X, y, epochs=20, batch_size=128, validation_split=0.1)
 Train on 14725 samples, validate on 1637 samples
Epoch 1/20
14725/14725 [==============================] - 23s - loss: 3.1173 - val_loss: 3.8415
Epoch 2/20
14725/14725 [==============================] - 20s - loss: 2.4202 - val_loss: 3.1512
Epoch 3/20
14725/14725 [==============================] - 20s - loss: 2.1302 - val_loss: 2.7191
Epoch 4/20
14725/14725 [==============================] - 20s - loss: 1.9435 - val_loss: 2.3341
Epoch 5/20
14725/14725 [==============================] - 20s - loss: 1.7966 - val_loss: 1.9877
Epoch 6/20
14725/14725 [==============================] - 20s - loss: 1.6621 - val_loss: 1.9349
Epoch 7/20
14725/14725 [==============================] - 20s - loss: 1.5190 - val_loss: 1.9632
Epoch 8/20
14725/14725 [==============================] - 20s - loss: 1.3925 - val_loss: 1.9735
"""

Train on 14725 samples, validate on 1637 samples
Epoch 1/13
14725/14725 [==============================] - 18s - loss: 3.1623 - val_loss: 3.9301
Epoch 2/13
14725/14725 [==============================] - 15s - loss: 2.4849 - val_loss: 3.2724
Epoch 3/13
14725/14725 [==============================] - 15s - loss: 2.2303 - val_loss: 2.7735
Epoch 4/13
14725/14725 [==============================] - 15s - loss: 2.0664 - val_loss: 2.3324
Epoch 5/13
14725/14725 [==============================] - 15s - loss: 1.9487 - val_loss: 2.0202
Epoch 6/13
14725/14725 [==============================] - 15s - loss: 1.8679 - val_loss: 1.9499
Epoch 7/13
14725/14725 [==============================] - 15s - loss: 1.7817 - val_loss: 1.9287
Epoch 8/13
14725/14725 [==============================] - 15s - loss: 1.6960 - val_loss: 1.9165
Epoch 9/13
14725/14725 [==============================] - 15s - loss: 1.6509 - val_loss: 1.9007
Epoch 10/13
14725/14725 [==============================] - 15s - loss: 1.5811 - val_los

"\nLSTM, BatchNorm\nTrain on 14929 samples, validate on 1659 samples\nEpoch 1/5\n14929/14929 [==============================] - 75s - loss: 3.0403 - val_loss: 3.4757\nEpoch 2/5\n14929/14929 [==============================] - 60s - loss: 2.3156 - val_loss: 2.9687\n\nLSTM\nTrain on 14929 samples, validate on 1659 samples\nEpoch 1/5\n14929/14929 [==============================] - 75s - loss: 3.1259 - val_loss: 2.7865\nEpoch 2/5\n14929/14929 [==============================] - 60s - loss: 2.6150 - val_loss: 2.3894\n\nCNN(5), LSTM  # faster, needs more epochs\nTrain on 14929 samples, validate on 1659 samples\nEpoch 1/5\n14929/14929 [==============================] - 42s - loss: 3.1579 - val_loss: 2.9987\nEpoch 2/5\n14929/14929 [==============================] - 26s - loss: 2.8874 - val_loss: 2.6994\nEpoch 3/5\n14929/14929 [==============================] - 26s - loss: 2.6220 - val_loss: 2.4879\nEpoch 4/5\n14929/14929 [==============================] - 26s - loss: 2.4309 - val_loss: 2.3942\nE

In [ ]:
model.fit(X, y, epochs=5, batch_size=128, validation_split=0.1)

In [ ]:
test_author_model.summary()

In [22]:
generate(model, diversity=0.7, text="this is some test text does it really matter what it says " * 30)

----- Generating with seed: "oes it really matter what it says this is some test text does it really matter what it says this is "
oes it really matter what it says this is some test text does it really matter what it says this is 

TypeError: object of type 'numpy.float64' has no len()

In [24]:
def generate(model, diversity=0.5, text=""):
    """Generate text from a model"""
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated = ''
    sentence = text[start_index: start_index + maxlen]
    generated += sentence
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)

    for i in range(5000):
        x = np.zeros((1, maxlen), dtype=np.int)
        for t, char in enumerate(sentence):
            try:
                x[0, t] = char_indices[char]
            except:
                print(sentence)
        preds = model.predict(x, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]
        generated += next_char
        sentence = sentence[1:] + next_char
        sys.stdout.write(next_char)
        sys.stdout.flush()
    return

In [26]:
generate(model, diversity=0.5, text="this is some test text does it really matter what it says " * 30)

----- Generating with seed: " what it says this is some test text does it really matter what it says this is some test text does "
 what it says this is some test text does it really matter what it says this is some test text does the chan store if the store in for Vis a singic and the places I nor for the store in find store is a gitter beat to the trist and the counders here to find. The sele the store with the storit and the chansic I was next the back the truck out. This is a git and the courder the trust and the ching store with the me to good and the next the countret to the this store is little back to the clourder the selsection for the stration's and my for probebles to have the store if friends out befould the chan net store with the park is can sele, I was not longer casteral friends. The store store is little this is this lace in findst and the inclure the selmen store in finds a befter the beet the car this store if the chan strater and closed to the counders. The strail a

KeyboardInterrupt: 